# Import Library

In [1]:
!pip install pymysql

     |████████████████████████████████| 51kB 3.4MB/s 


In [173]:
import json
import urllib
import pandas as pd
from urllib.parse import urlencode, quote_plus, unquote, quote
from urllib.request import urlopen
from pandas.io.json import json_normalize
from sqlalchemy import create_engine
import pymysql
import datetime as dt
import sklearn
from sklearn.preprocessing import LabelEncoder

pymysql.install_as_MySQLdb()
import MySQLdb

pd.options.display.max_colwidth = 100

# Dataframe make function

In [3]:
service_key = 'SE0dHlfRrFs9F7Z8K6y6qmYRqrAFPtIrJnMdsrJLTMm1y5JvTVKl5Z8JpG4%2BBcvj%2Bv9kS1d1%2B8gQBmmxcMqMXg%3D%3D'

## Election Code dataframe make function

In [4]:
def get_election_df(pg_num = 1, num_row = 100):
  election_base_url = 'http://apis.data.go.kr/9760000/CommonCodeService/getCommonSgCodeList'

  page_no = str(pg_num)
  num_of_rows = str(num_row)                                  # maximum 34 datas

  queryParams = '?' + urlencode({
      quote_plus('pageNo') : page_no,
      quote_plus('numOfRows') : num_of_rows,
      quote_plus('resultType') : 'json',
      quote_plus('ServiceKey') : service_key 
  })

  API_election_code_url = election_base_url + unquote(queryParams)

  # Election code load

  '''
  < election code >
  (0)대표선거명               
  (1)대통령,(2)국회의원,(3)시도지사,(4)구시군장,(5)시도의원,(6)구시군의회의원
  (7)국회의원비례대표,(8)광역의원비례대표,(9)기초의원비례대표,(10)교육의원,(11)교육감
  '''

  response = urlopen(API_election_code_url)
  json_str = response.read()

  json_object = json.loads(json_str)

  body = [json_object['getCommonSgCodeList']['item']]

  election_code_data = pd.json_normalize(json_object['getCommonSgCodeList']['item'])
  
  return election_code_data

## Candidate Info dataframe make function

In [5]:
def get_candidate_df(sgId, sgTypecode, num_of_row=1000):
  candidate_base_url = 'http://apis.data.go.kr/9760000/PofelcddInfoInqireService/getPofelcddRegistSttusInfoInqire'

  queryParams = '?' + urlencode({
      quote_plus('numOfRows') : str(num_of_row),        # optional
      quote_plus('sgId') : str(sgId),                   # load from election code
      quote_plus('sgTypecode') : str(sgTypecode),       # load from election code
      quote_plus('sggName') : '',                       # optional
      quote_plus('sdName') : '',                        # optional
      quote_plus('resultType') : 'json',                # must set JSON
      quote_plus('ServiceKey') : service_key
  })

  API_candidate_url = candidate_base_url + unquote(queryParams)

  # candidate info load

  response = urlopen(API_candidate_url)
  json_str = response.read().decode('utf-8')

  json_object = json.loads(json_str)

  body = [json_object['getPofelcddRegistSttusInfoInqire']['item']]

  candidate_data = pd.json_normalize(json_object['getPofelcddRegistSttusInfoInqire']['item'])

  candidate_data.rename(columns={'HUBOID' : 'cnddtId', 'JD_NAME': 'partyName'}, inplace=True)

  return candidate_data

## Elction Promise dataframe make function

In [6]:
def get_promise_df(sgId, sgTypecode, hudoId):
  # 선거종료 후 데이터가 갱신되는 시점부터는 당선인의 공약 정보만 제공

  promise_base_url = 'http://apis.data.go.kr/9760000/ElecPrmsInfoInqireService/getCnddtElecPrmsInfoInqire'

  queryParams = '?' + urlencode({
      quote_plus('sgId') : str(sgId),                   # load from election code
      quote_plus('sgTypecode') : str(sgTypecode),       # load from election code
      quote_plus('cnddtId') : str(hudoId),              # parameter name is cnddtId but DB column name is HUBOID
      quote_plus('resultType') : 'json',                # must set JSON
      quote_plus('ServiceKey') : service_key
  })

  API_promise_url = promise_base_url + unquote(queryParams)

  # promise load

  response = urlopen(API_promise_url)
  json_str = response.read().decode('utf-8')

  json_object = json.loads(json_str)

  body = [json_object['getCnddtElecPrmsInfoInqire']['item']]

  promise_data = pd.json_normalize(json_object['getCnddtElecPrmsInfoInqire']['item'])

  return promise_data

## Polls Info dataframe make function


### Pre polls info dataframe make function

In [7]:
def get_prepoll_df(sgId, sdName):
  prepoll_base_url = 'http://apis.data.go.kr/9760000/PolplcInfoInqireService2/getPrePolplcOtlnmapTrnsportInfoInqire'

  queryParams = '?' + urlencode({
      quote_plus('sgId') : str(sgId),                   # load from election code
      quote_plus('sdName') : quote(sdName),             # must input parameter, query parma is korean, so wrap the quote
      quote_plus('resultType') : 'json',                # must set JSON
      quote_plus('ServiceKey') : service_key
  })

  API_prepoll_url = prepoll_base_url + unquote(queryParams)

  response = urlopen(API_prepoll_url)
  json_str = response.read().decode('utf-8')

  json_object = json.loads(json_str)

  body = [json_object['getPrePolplcOtlnmapTrnsportInfoInqire']['item']]

  prepoll_data = pd.json_normalize(json_object['getPrePolplcOtlnmapTrnsportInfoInqire']['item'])
  return prepoll_data

### Main polls info dataframe make function

In [8]:
def get_mainpoll_df(sgId, sdName):
  poll_base_url = 'http://apis.data.go.kr/9760000/PolplcInfoInqireService2/getPolplcOtlnmapTrnsportInfoInqire'

  queryParams = '?' + urlencode({
      quote_plus('sgId') : str(sgId),                   # load from election code
      quote_plus('sdName') : quote(sdName),             # must input parameter, query parma is korean, so wrap the quote
      quote_plus('resultType') : 'json',                # must set JSON
      quote_plus('ServiceKey') : service_key
  })

  API_poll_url = poll_base_url + unquote(queryParams)

  response = urlopen(API_poll_url)
  json_str = response.read().decode('utf-8')

  json_object = json.loads(json_str)

  body = [json_object['getPolplcOtlnmapTrnsportInfoInqire']['item']]

  poll_data = pd.json_normalize(json_object['getPolplcOtlnmapTrnsportInfoInqire']['item'])
  return poll_data

# DataFrame load test

## Election code load

In [174]:
get_election_df().head()

,NUM,SG_ID,SG_NAME,SG_TYPECODE,SG_VOTEDATE
0,1,20160413,제20대 국회의원선거,0,20160413
1,2,20160413,국회의원선거,2,20160413
2,3,20160413,구·시·군의장선거,4,20160413
3,4,20160413,시·도의회의원선거,5,20160413
4,5,20160413,구·시·군의회의원선거,6,20160413


## Cadidate info load

In [ ]:
get_candidate_df(20210407, 3).head()

,NUM,SG_ID,SG_TYPECODE,cnddtId,SGG_NAME,SD_NAME,WIW_NAME,GIHO,GIHO_SANGSE,partyName,NAME,HANJA_NAME,GENDER,BIRTHDAY,AGE,ADDR,JOB_ID,JOB,EDU_ID,EDU,CAREER1,CAREER2,STATUS
0,1,20210407,3,100138295,서울특별시,서울특별시,,1,,더불어민주당,박영선,朴映宣,여,19600122,61,서울특별시 서대문구 연희로27나길,75,정당인,221,서강대학교 언론대학원 졸업 (문학석사),(전) 중소벤처기업부 장관,"(전) 17대,18대,19대,20대 국회의원",등록
1,2,20210407,3,100138272,서울특별시,서울특별시,,2,,국민의힘,오세훈,吳世勳,남,19610104,60,서울특별시 광진구 능동로4길,85,변호사,221,고려대학교 법학대학원 졸업(법학박사),"(전) 제33, 34대 서울특별시장",(전) 16대 국회의원,등록
2,3,20210407,3,100138291,서울특별시,서울특별시,,4,,국민의당,안철수,安哲秀,남,19620122,59,서울특별시 노원구 동일로242나길,75,정치인,221,서울대학교 대학원 의학박사,"(전)제19대, 제20대 국회의원",(현)국민의당 대표,사퇴
3,4,20210407,3,100138218,서울특별시,서울특별시,,6,,기본소득당,신지혜,申智惠,여,19871022,33,서울특별시 은평구 통일로72길,75,정당인,68,이화여자대학교 사회학과 졸업(복수전공 : 여성학),(현)기본소득당 상임대표,(전)포이동 인연공부방 교사,등록
4,5,20210407,3,100138226,서울특별시,서울특별시,,7,,국가혁명당,허경영,許京寧,남,19470713,73,서울특별시 종로구 북촌로5나길,225,강연업,68,한국방송통신대학교 법학과 법학사,"(전)15, 17대 대통령 후보 출마",(현)국가혁명당 명예대표,등록


## Promise load

In [ ]:
# 당선인이 나온 후엔 당선인 공약만 확인 가능
data = get_promise_df(20210407, 3, 100138272)
data

,num,sgId,sgTypecode,cnddtId,sggName,sidoName,wiwName,partyName,krName,cnName,prmsCnt,prmsOrd1,prmsRealmName1,prmsTitle1,prmmCont1,prmsOrd2,prmsRealmName2,prmsTitle2,prmmCont2,prmsOrd3,prmsRealmName3,prmsTitle3,prmmCont3,prmsOrd4,prmsRealmName4,prmsTitle4,prmmCont4,prmsOrd5,prmsRealmName5,prmsTitle5,prmmCont5,prmsOrd6,prmsRealmName6,prmsTitle6,prmmCont6,prmsOrd7,prmsRealmName7,prmsTitle7,prmmCont7,prmsOrd8,prmsRealmName8,prmsTitle8,prmmCont8,prmsOrd9,prmsRealmName9,prmsTitle9,prmmCont9,prmsOrd10,prmsRealmName10,prmsTitle10,prmmCont10
0,1,20210407,3,100138272,서울특별시,서울특별시,,국민의힘,오세훈,吳世勳,5,1,,스피드 주택공급,□ 목 표\n ○ 주택공급 활성화로 부동산 시장 안정화\n \n□ 이행방법\n ○ 1년내 서울시 도시계획규제 혁파 \n - 서울시 용적률 규제완화(주거지역 용적률 ...,2,,스피드 교통,□ 목 표\n ○ 교통소외지역 연결 및 편의증대\n \n□ 이행방법\n ○ 1년내 시민불편공사 완료\n - 다리공사 국내 최장기록(11년4개월) 달성한 월드...,3,,균형발전 서울,"□ 목 표\n ○ 동북권, 서남권, 서북권 등 권역별 일자리, 상업, 교통, 교육 등 핵심시설을 유치하여 지역균형발전 달성 \n \n□ 이행방법\n ○ 서북권: 신...",4,,1인가구 안심특별대책본부 설치,"□ 목 표\n ○ 안전, 질병, 빈곤(저소득), 외로움(우울), 주거 등 1인가구 5대 불안해소\n \n□ 이행방법\n ○ 시장 직속 임시조직 : 조례 개정,...",5,,청춘이 밥 먹여준다!,"□ 목 표\n ○ 취업교육, 취업과 창업, 자산형성, 내집마련으로 이어지는 실질적이고 전방위적인 터널탈출 프로젝트\n \n□ 이행방법\n ○ 4차산업형 청년 취업사관...",,,,,,,,,,,,,,,,,,,,


In [ ]:
data['prmmCont1']

0    □ 목 표\n  ○ 주택공급 활성화로 부동산 시장 안정화\n \n□ 이행방법\n  ○ 1년내 서울시 도시계획규제 혁파 \n    - 서울시 용적률 규제완화(주거지역 용적률 ...
Name: prmmCont1, dtype: object

## Poll info load

### Pre poll info load

In [ ]:
get_prepoll_df(20210407, '서울특별시').head()

,NUM,SG_ID,EV_PS_NAME,SD_NAME,WIW_NAME,EMD_NAME,EV_ORDER,PLACE_NAME,ADDR,FLOOR
0,1,20210407,가락1동사전투표소,서울특별시,송파구,가락1동,1,"가락1동주민센터(3층, 대강당)",서울특별시 송파구 양재대로 925 (가락동),3층
1,2,20210407,가락2동사전투표소,서울특별시,송파구,가락2동,1,"가락2동주민센터(2층, 다목적실)",서울특별시 송파구 중대로20길 6 (가락동),2층
2,3,20210407,가락본동사전투표소,서울특별시,송파구,가락본동,1,"가락본동주민센터(2층, 다목적방)",서울특별시 송파구 송파대로28길 39 (가락동),2층
3,4,20210407,가리봉동사전투표소,서울특별시,구로구,가리봉동,1,"가리봉동주민센터(1층, 회의실)",서울특별시 구로구 우마2길 35 (가리봉동),1층
4,5,20210407,가산동사전투표소,서울특별시,금천구,가산동,1,가산동주민센터 문화관람실(2층),서울특별시 금천구 가산로 129 (가산동),2층


### Main poll info load

In [ ]:
get_mainpoll_df(20210407, '서울특별시').head()

,NUM,SG_ID,PS_NAME,SD_NAME,WIW_NAME,EMD_NAME,PLACE_NAME,ADDR,FLOOR
0,1,20210407,청룡동제1투,서울특별시,관악구,청룡동,"봉림중학교(1층, 무용실)",서울특별시 관악구 장군봉길 72 (봉천동),1층
1,2,20210407,청림동제1투,서울특별시,관악구,청림동,"청림동주민센터(1층, 주차장)",서울특별시 관악구 관악로40길 39 (봉천동),1층
2,3,20210407,청운효자동제1투,서울특별시,종로구,청운효자동,"청운초등학교(2층, 체육관)",서울특별시 종로구 자하문로 105 (청운동),2층
3,4,20210407,청파동제1투,서울특별시,용산구,청파동,"청파도서관·청파동자치회관 별관(3층, 대강당)",서울특별시 용산구 청파로93길 27 (서계동),3층이상
4,5,20210407,충현동제1투,서울특별시,서대문구,충현동,"충현동주민센터(2층, 대회의실)",서울특별시 서대문구 북아현로22길 38 (북아현동),2층


# Data Preprocessing

## election_code data preprocessing

In [96]:
def election_code_preprocessing():
  data = get_election_df()
  data = data.drop('NUM', axis=1)
  data.columns = ['sgId', 'sgName', 'sgTypecode', 'sgVotedate']
  election_code = data
  election_code = election_code.astype({'sgId':int, 'sgTypecode':int, 'sgName':str})
  election_code.set_index('sgId', inplace=True)

  return election_code

In [113]:
election_code = election_code_preprocessing()
data = election_code.reset_index()

In [114]:
election_code.head()

,sgName,sgTypecode,sgVotedate
sgId,,,
20160413,제20대 국회의원선거,0,20160413
20160413,국회의원선거,2,20160413
20160413,구·시·군의장선거,4,20160413
20160413,시·도의회의원선거,5,20160413
20160413,구·시·군의회의원선거,6,20160413


## Candidate data preprocessing

In [117]:
election_data = data[['sgId', 'sgTypecode']]
election_data.head()

,sgId,sgTypecode
0,20160413,0
1,20160413,2
2,20160413,4
3,20160413,5
4,20160413,6


### Concat the all candidate df

In [153]:
candidate_df = pd.DataFrame()

for i in range(len(election_data)):
  curr_id = election_data['sgId'][i]
  curr_code = election_data['sgTypecode'][i]
  curr_df = pd.DataFrame()

  if curr_code != 0:
    curr_df = get_candidate_df(curr_id, curr_code)
  
  candidate_df = pd.concat([candidate_df, curr_df], ignore_index=True)

In [185]:
cand_age = candidate_df['AGE']
candidate = candidate_df.drop(['NUM', 'GIHO_SANGSE', 'HANJA_NAME', 'JOB_ID', 'EDU_ID', 'EDU', 'CAREER1', 'CAREER2', 'partyName', 'AGE'], axis=1)

In [186]:
candidate.columns = ['sgId', 'sgTypecode', 'cnddtId', 'sggName', 'sdName', 'wiwName', 'giho', 'name', 'gender', 'birthday','address', 'job', 'status']

In [187]:
candidate.tail()

,sgId,sgTypecode,cnddtId,sggName,sdName,wiwName,giho,name,gender,birthday,address,job,status
6966,20210407,6,100138297,보성군다선거구,전라남도,보성군,7,윤정재,남,19690412,전라남도 보성군 복내면 송재로,자영업,등록
6967,20210407,6,100138344,의령군다선거구,경상남도,의령군,2,차성길,남,19610519,경상남도 의령군 정곡면 법정로8길,무직,등록
6968,20210407,6,100138337,의령군다선거구,경상남도,의령군,6,윤병열,남,19590514,경상남도 의령군 지정면 함의로,농업,등록
6969,20210407,6,100138310,함안군다선거구,경상남도,함안군,1,조호기,남,19740911,경상남도 함안군 칠원읍 삼호길,자영업,등록
6970,20210407,6,100138289,함안군다선거구,경상남도,함안군,2,황철용,남,19670328,경상남도 함안군 칠원읍 경남대로,농업인,등록


In [189]:
le = LabelEncoder()

for category in ['gender', 'status']:
  candidate[category] = le.fit_transform(candidate[category])

candidate.tail()

,sgId,sgTypecode,cnddtId,sggName,sdName,wiwName,giho,name,gender,birthday,address,job,status
6966,20210407,6,100138297,보성군다선거구,전라남도,보성군,7,윤정재,0,19690412,전라남도 보성군 복내면 송재로,자영업,0
6967,20210407,6,100138344,의령군다선거구,경상남도,의령군,2,차성길,0,19610519,경상남도 의령군 정곡면 법정로8길,무직,0
6968,20210407,6,100138337,의령군다선거구,경상남도,의령군,6,윤병열,0,19590514,경상남도 의령군 지정면 함의로,농업,0
6969,20210407,6,100138310,함안군다선거구,경상남도,함안군,1,조호기,0,19740911,경상남도 함안군 칠원읍 삼호길,자영업,0
6970,20210407,6,100138289,함안군다선거구,경상남도,함안군,2,황철용,0,19670328,경상남도 함안군 칠원읍 경남대로,농업인,0


encoding the gender data 0 / 1

In [220]:
candidate[candidate['giho'] == ''] = 1

giho null value processing

In [223]:
candidate = candidate.astype({'sgId':int, 'sgTypecode':int, 'cnddtId':int, 'gender':int, 'status':int, 'giho':int})
candidate.set_index('sgId', inplace=True)

In [229]:
candidate.reset_index()

,sgId,sgTypecode,cnddtId,sggName,sdName,wiwName,giho,name,gender,birthday,address,job,status
0,20160413,2,100118613,종로구,서울특별시,종로구,1,오세훈,0,19610104,,변호사,0
1,20160413,2,100119321,종로구,서울특별시,종로구,2,정세균,0,19500926,,국회의원,0
2,20160413,2,100119833,종로구,서울특별시,종로구,3,박태순,0,19630228,,사회갈등연구소 대표이사,0
3,20160413,2,100120092,종로구,서울특별시,종로구,4,윤공규,0,19630421,,정당인,0
4,20160413,2,100120147,종로구,서울특별시,종로구,5,김한울,0,19781216,,종로구 지역 활동가,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6966,20210407,6,100138297,보성군다선거구,전라남도,보성군,7,윤정재,0,19690412,전라남도 보성군 복내면 송재로,자영업,0
6967,20210407,6,100138344,의령군다선거구,경상남도,의령군,2,차성길,0,19610519,경상남도 의령군 정곡면 법정로8길,무직,0
6968,20210407,6,100138337,의령군다선거구,경상남도,의령군,6,윤병열,0,19590514,경상남도 의령군 지정면 함의로,농업,0
6969,20210407,6,100138310,함안군다선거구,경상남도,함안군,1,조호기,0,19740911,경상남도 함안군 칠원읍 삼호길,자영업,0


In [227]:
candidate.head()

,sgTypecode,cnddtId,sggName,sdName,wiwName,giho,name,gender,birthday,address,job,status
sgId,,,,,,,,,,,,
20160413,2,100118613,종로구,서울특별시,종로구,1,오세훈,0,19610104,,변호사,0
20160413,2,100119321,종로구,서울특별시,종로구,2,정세균,0,19500926,,국회의원,0
20160413,2,100119833,종로구,서울특별시,종로구,3,박태순,0,19630228,,사회갈등연구소 대표이사,0
20160413,2,100120092,종로구,서울특별시,종로구,4,윤공규,0,19630421,,정당인,0
20160413,2,100120147,종로구,서울특별시,종로구,5,김한울,0,19781216,,종로구 지역 활동가,0


## Polls data preprocessing

### pre_polls data preprocessing

### main_polls data preprocessing

# MySQL link to GCP

## GCP MySQL connection

In [31]:
host = '34.64.72.201'
user = 'root'
pw = 'root'
db = 'election'
port = 3306

try:
    conn = pymysql.connect(host=host, port=port, user=user, password=pw, db=db)
    engine = create_engine('mysql+mysqldb://'+user+':'+pw+'@'+host+':3306/'+db, encoding='utf-8')

except Exception as err:
    print('Connection Failed (Error msg : ' + str(err) + ')')
else:
    print('Connection  Success!')

Connection  Success!


In [32]:
connection = engine.connect()
connection

# Queries & data insert

## CREATE TABLE

In [ ]:
create_election_code = """CREATE TABLE election_code (  
    sgId INT NOT NULL,
    sgTypecode INT NOT NULL,
    sgName INT NOT NULL,
    sgVotedate TIMESTAMP NOT NULL,
    PRIMARY KEY (sgId, sgTypecode)
) default charset utf8;"""

In [ ]:
create_pre_polls = """CREATE TABLE pre_polls (  
    sgId INT NOT NULL,
    sdName VARCHAR(30) NOT NULL ,
    wiwName VARCHAR(30) NOT NULL ,
    emdName VARCHAR(50) NOT NULL ,
    evOrder INT NOT NULL ,
    evPsName VARCHAR(80) NOT NULL ,
    placeName VARCHAR(100) NOT NULL ,
    `address` VARCHAR(150) NOT NULL ,
    `floor` INT NOT NULL ,
    PRIMARY KEY(evPsName) ,
    FOREIGN KEY(sgId) REFERENCES election_code(sgId)
) default charset utf8;"""

In [ ]:
create_main_polls = """CREATE TABLE main_polls (  
    sgId INT NOT NULL,
    sdName VARCHAR(30) NOT NULL ,
    wiwName VARCHAR(30) NOT NULL ,
    emdName VARCHAR(50) NOT NULL ,
    PsName VARCHAR(80) NOT NULL ,
    placeName VARCHAR(100) NOT NULL ,
    `address` VARCHAR(150) NOT NULL ,
    `floor` INT NOT NULL ,
    PRIMARY KEY(PsName) ,
    FOREIGN KEY(sgId) REFERENCES election_code(sgId)
) default charset utf8;"""

In [ ]:
create_politician = """CREATE TABLE politician (
    num INT NOT NULL ,
    `name` VARCHAR(20) NOT NULL ,
    imageLink TEXT ,
    sggName VARCHAR(35) NOT NULL ,
    deptCode INT NOT NULL ,
    memTitle LONGTEXT ,
    birthday TIMESTAMP NOT NULL ,
    age INT NOT NULL ,
    PRIMARY KEY (num)
) default charset utf8;"""

In [ ]:
create_candidate = """CREATE TABLE candidate (
    cnddtId INT NOT NULL ,
    num INT NOT NULL ,
    sgId INT NOT NULL ,
    sgTypecode INT NOT NULL ,
    sggName VARCHAR(35) NOT NULL ,
    sdName VARCHAR(30) NOT NULL ,
    wiwName VARCHAR(30) NOT NULL ,
    giho INT NOT NULL ,
    `name` VARCHAR(20) NOT NULL ,
    gender TINYINT NOT NULL ,
    `address` VARCHAR(100) ,
    job VARCHAR(100) ,
    `status` TINYINT NOT NULL ,
    birthday TIMESTAMP NOT NULL ,
    PRIMARY KEY (cnddtId),
    FOREIGN KEY (num) REFERENCES politician (num),
    FOREIGN KEY (sgId, sgTypecode) REFERENCES election_code (sgId, sgTypecode)
) default charset utf8;"""

In [ ]:
create_promise = """CREATE TABLE promise (
    prmIdx INT AUTO_INCREMENT NOT NULL ,
    cnddtId INT NOT NULL ,
    prmOrd INT ,
    prmTitle VARCHAR(100) ,
    prmCont LONGTEXT,
    PRIMARY KEY (prmIdx),
    FOREIGN KEY (cnddtId) REFERENCES candidate (cnddtId)
);"""

## Data insert

### election_code insert

In [47]:
election_code.to_sql(name='election_code', con=engine, if_exists='append')

### candidate insert

In [ ]:
candidate.to_sql(name='candidate')